## Oracle Cloud Instance POC


In [1]:
import os
import json
from pathlib import Path
from typing import Dict
from dataclasses import dataclass

import bravado
from bravado.client import SwaggerClient, CallableOperation
# from bravado.requests_client import RequestsClient
from gc3_query.lib.gc3_bravado.requests_client import OCRequestsClient
from bravado.requests_client import RequestsResponseAdapter
from bravado.swagger_model import load_file
from bravado_core.exception import MatchingResponseNotFound
from bravado.exception import HTTPBadRequest
from bravado.http_future import HttpFuture

from tinydb import TinyDB, Query

# from prettyprinter import pprint, pformat
from pprint import pprint, pformat

## https://medium.com/@betz.mark/validate-json-models-with-swagger-and-bravado-5fad6b21a825
# Validate json models with swagger and bravado
from bravado_core.spec import Spec

In [2]:
### B01
import toml
from pathlib import Path

from gc3_query.lib import *
from gc3_query.lib import BASE_DIR
from gc3_query.lib import List, Optional, Any, Callable, Dict, Tuple, Union, Set, Generator, Path

from gc3_query.lib.gc3_config import GC3Config, IDMCredential
from gc3_query.lib.iaas_classic.iaas_requests_http_client import IaaSRequestsHTTPClient
from gc3_query.lib.iaas_classic import IaaSServiceBase, API_SPEC_DIR, IaaSRequestsHTTPClient
from gc3_query.lib.iaas_classic.instances import Instances



In [3]:
### B01
ORACLE_VPN_CONNECTED=False
ASDF='ASDF'

In [4]:
### B01
service = 'Instances'
idm_domain = 'gc30003'
gc3_config = GC3Config()
service_cfg = gc3_config.iaas_classic.services[service]
idm_cfg = gc3_config.idm.domains[idm_domain]
http_client: IaaSRequestsHTTPClient = IaaSRequestsHTTPClient(idm_cfg=idm_cfg)

In [5]:
# from secrets import opc_username, opc_password
import keyring

opc_username = "eric.harris@oracle.com"
opc_password = keyring.get_password("gc3", "gc30003")
print(f"opc_username={opc_username}, opc_password={opc_password}")

opc_username=eric.harris@oracle.com, opc_password=None


In [6]:
### B01
opc_password = keyring.get_password(service_name="gc3@gc30003", username="eric.harris@oracle.com")
print(f"opc_username={opc_username}, opc_password={opc_password}")

opc_username=eric.harris@oracle.com, opc_password=V@nadium123!


In [7]:
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}\n')


### Username/pass setup
idm_domain_username = f'/Compute-{idm_domain_name}/{opc_username}'
idm_service_instance_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# username = traditional_iaas_username
username = idm_service_instance_username
# basic_auth_cred = _basic_auth_str(username, opc_password)


iaas_rest_endpoint: https://compute.uscom-central-1.oraclecloud.com
iaas_auth_endpoint: https://compute.uscom-central-1.oraclecloud.com/authenticate/



In [8]:
json_data = {"user": username, "password": opc_password}
print(f'idm_domain_username: {idm_domain_username}')
print(f'idm_service_instance_username: {idm_service_instance_username}')
print(f'username: {username}')

idm_domain_username: /Compute-gc30003/eric.harris@oracle.com
idm_service_instance_username: /Compute-587626604/eric.harris@oracle.com
username: /Compute-587626604/eric.harris@oracle.com


In [9]:
proxies = {
  'http': 'http://www-proxy-ash7.us.oracle.com:80',
  'https': 'https://www-proxy-ash7.us.oracle.com:80',
}

#### Create DB to store Instance data

In [10]:
instances_db = TinyDB('instances.tdb.json')

####  Authenticate and get header token


In [11]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+directory+json'),
                ])
requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)


# print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
requests_client.session.headers.update(headers)
# print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")

response = requests_client.session.post(url=iaas_auth_endpoint, json=json_data)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
#     print(f"\nSession Cookies: {requests_client.session.cookies}")
#     print(f"\nResponse Headers['Set-Cookie']: {response.headers['Set-Cookie']}")
else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

# print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
cookie_header = {'Cookie': response.headers['Set-Cookie']}
# print(f"cookie_header: {cookie_header}\n")
requests_client.session.headers.update(cookie_header)
# print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")


Response OK: True, Status Code: 204, URL: https://compute.uscom-central-1.oraclecloud.com/authenticate/
Auth request succeess.



In [12]:
## Update the swagger spec to use https
spec_file_path = Path().joinpath('open_api_definitions/iaas_instances.json').resolve()
spec_dict = load_file(spec_file_path)
spec_dict['schemes'] = ['https']


In [13]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, application/oracle-compute-v3+directory+json, json, text/html')
                ])

# headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
#                 ('Accept', 'application/oracle-compute-v3+json, json, text/html')])

In [14]:
requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)
# requests_client.session.proxies.update(proxies)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         config={'also_return_response': True,
                                                 'validate_responses': True,
                                                 'validate_requests': True,
                                                 'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
#                                          config={'also_return_response': True,
#                                                  'validate_responses': False,
#                                                  'validate_requests': False,
#                                                  'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [15]:
for operation_name in dir(instances_resource):
    callable_operation = getattr(instances_resource, operation_name)
    print(f"{callable_operation.operation.operation_id}\t\t http_method={callable_operation.operation.http_method}\tpath={callable_operation.operation.path_name}")
    
    

deleteInstance		 http_method=delete	path=/instance/{name}
discoverInstance		 http_method=get	path=/instance/{container}
discoverRootInstance		 http_method=get	path=/instance/
getInstance		 http_method=get	path=/instance/{name}
listInstance		 http_method=get	path=/instance/{container}/
updateInstance		 http_method=put	path=/instance/{name}


In [16]:
callable_operation

In [17]:
@dataclass
class GetOpReturn:
    operation_details: Dict[str, Any]
    operation_response: RequestsResponseAdapter


def get_op(operation_id: str, **wkargs) -> Tuple[Dict[str, Any], RequestsResponseAdapter]:
    print(f"Operation {operation_id} starting.")
    try:
        operation_future = getattr(instances_resource, operation_id)(**wkargs)
        url = operation_future.future.request.url
        print(f"REST url for {operation_id}: {url}")
        operation_result, operation_response = operation_future.result()
    except HTTPBadRequest:
        print("Request failed for {operation_id}! ")
        print(f"URL: {operation_future.future.request.url}")
        raise
    operation_details = json.loads(operation_result)
    return GetOpReturn(operation_details,operation_response )
#     print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
#     print(f"Operation {operation_id} finished.\n")


In [18]:
operation_id = "discoverRootInstance"
print(f"Operation {operation_id} starting.")
try:
    operation_future = getattr(instances_resource, operation_id)()
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverRootInstance starting.
REST url for discoverRootInstance: https://compute.uscom-central-1.oraclecloud.com/instance/

discoverRootInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/:
 {'result': ['/Compute-587626604/']}

Operation discoverRootInstance finished.



In [19]:
### B01

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
http_future = instances.bravado_service_operations.discoverRootInstance()
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code





In [20]:
### B01
service_response

In [21]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")









operation_id=discoverRootInstance
service_response_status_code=200
http_response=200 OK
b01_result={"result": ["/Compute-587626604/"]}
s01_operation_result={"result": ["/Compute-587626604/"]}
is_same=True
ASDF=ASDF
ASDF=ASDF
ASDF=ASDF


In [22]:
## discoverInstance
## container="Compute-587626604/"
operation_id = "discoverInstance"

print(f"Operation {operation_id} starting.")
try:
    # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604
    container = operation_details['result'][0].lstrip('/').rstrip('/')
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com
    # container = f"{container}/{opc_username}"
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
    container = f"{container}/"
    container = "Compute-587626604/"
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverInstance starting.
container: Compute-587626604/
REST url for discoverInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/

discoverInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/:
 {'result': ['/Compute-587626604/dhiru.vallabhbhai@oracle.com/',
            '/Compute-587626604/eric.harris@oracle.com/',
            '/Compute-587626604/manjunath.udupa@oracle.com/',
            '/Compute-587626604/mayurnath.gokare@oracle.com/',
            '/Compute-587626604/ramesh.dadhania@oracle.com/',
            '/Compute-587626604/seetharaman.nandyal@oracle.com/',
            '/Compute-587626604/siva.subramani@oracle.com/',
            '/Compute-587626604/tarek.orfaly@oracle.com/']}

Operation discoverInstance finished.



In [23]:
## discoverInstance
## container="Compute-587626604/eric.harris@oracle.com/"

operation_id = "discoverInstance"
print(f"Operation {operation_id} starting.")
try:
    # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604
#     container = operation_details['result'][0].lstrip('/').rstrip('/')
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com
    # container = f"{container}/{opc_username}"
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
    container = f"{container}/{opc_username}/"
    container="Compute-587626604/eric.harris@oracle.com/"
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverInstance starting.
container: Compute-587626604/eric.harris@oracle.com/
REST url for discoverInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/

discoverInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/:
 {'result': ['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/',
            '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-PSFTHR92/',
            '/Compute-587626604/eric.harris@oracle.com/dbaas/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-naac-cdmt-d07-lws/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d03-lws/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d04-02-essbase-bi/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d04-essbase-bi/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-

In [24]:
### B01
## discoverInstance
## container="Compute-587626604/eric.harris@oracle.com/"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_container=instances.idm_user_container_name

print(f"b01_container={b01_container}")
print(f"s01_container={container}")
is_same_container = b01_container==container

print(f"is_same_container={is_same_container}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

b01_container=Compute-587626604/eric.harris@oracle.com/
s01_container=Compute-587626604/eric.harris@oracle.com/
is_same_container=True
ASDF=ASDF
ASDF=ASDF
ASDF=ASDF


In [25]:
### B01

http_future = instances.bravado_service_operations.discoverInstance(container=b01_container)
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [26]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")





operation_id=discoverInstance
service_response_status_code=200
http_response=200 OK
b01_result={"result": ["/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/", "/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-PSFTHR92/", "/Compute-587626604/eric.harris@oracle.com/dbaas/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-cdmt-d07-lws/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d03-lws/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d04-02-essbase-bi/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d04-essbase-bi/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d05-lws/", "/Compute-587626604/eric.harris@oracle.com/gc3_naac_soar_ebs1226_demo_01/", "/Compute-587626604/eric.harris@oracle.com/paas/"]}
s01_operation_result={"result": ["/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/", "/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-PSFTHR92/", "/Compute-587626604/eric.harris@oracle.com/dbaas/", "/Compu

In [27]:
operation_id = "listInstance"
print(f"Operation {operation_id} starting.")
try:
    # container = operation_details['result'][0].lstrip('/').rstrip('/')
    # container = 'Compute-587626604/eric.harris@oracle.com'
    container = 'Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")

Operation listInstance starting.
container: Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1
REST url for listInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/

listInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/:
 {'result': ['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7']}

Operation listInstance finished.


In [28]:
operation_details

{'result': ['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7']}

In [29]:
operation_details['result']

['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7']

In [30]:
### B01
## listInstance
## container: "Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_container="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1"

print(f"b01_container={b01_container}")
print(f"s01_container={container}")
is_same_container = b01_container==container

print(f"is_same_container={is_same_container}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

b01_container=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1
s01_container=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1
is_same_container=True
ASDF=ASDF
ASDF=ASDF
ASDF=ASDF


In [32]:
### B01

http_future = instances.bravado_service_operations.listInstance(container=b01_container)
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [33]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")



operation_id=listInstance
service_response_status_code=200
http_response=200 OK
b01_result={"result": ["/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"]}
s01_operation_result={"result": ["/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"]}
is_same=True
ASDF=ASDF
ASDF=ASDF
ASDF=ASDF


In [ ]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])


requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)
# requests_client.session.proxies.update(proxies)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         # config={'also_return_response': True,
                                         #         'validate_responses': True,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         config={'also_return_response': True,
                                                 'validate_responses': False,
                                                 'validate_requests': False,
                                                 'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [ ]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")

In [ ]:
operation_details

In [ ]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
# print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")



<pre>

      "get" : {
        "tags" : [ "Instances" ],
        "summary" : "Retrieve Details of an Instance",
        "description" : "Retrieves details of the specified instance.<p><b>Required Role: </b>To complete this task, you must have the <code>Compute_Monitor</code> or <code>Compute_Operations</code> role. If this role isn't assigned to you or you're not sure, then ask your system administrator to ensure that the role is assigned to you in Oracle Cloud My Services. See <a target=\"_blank\" href=\"http://www.oracle.com/pls/topic/lookup?ctx=stcomputecs&id=MMOCS-GUID-54C2E747-7D5B-451C-A39C-77936178EBB6\">Modifying User Roles</a> in <em>Managing and Monitoring Oracle Cloud</em>.",
        "operationId" : "getInstance",
        "responses" : {
          "200" : {
            "headers" : {
              "set-cookie" : {
                "type" : "string",
                "description" : "The cookie value is returned if the session is extended"
              }
            },
            "description" : "OK. See <a class=\"xref\" href=\"Status%20Codes.html\">Status Codes</a> for information about other possible HTTP status codes.",
            "schema" : {
              "$ref" : "#/definitions/Instance-response"
            }
          }
        },
        "consumes" : [ "application/oracle-compute-v3+json" ],
        "produces" : [ "application/oracle-compute-v3+json" ],
        "parameters" : [ {
          "name" : "name",
          "in" : "path",
          "description" : "<p>Multipart name of the object.",
          "required" : true,
          "type" : "string"
        }, {
          "name" : "Cookie",
          "in" : "header",
          "type" : "string",
          "description" : "The Cookie: header must be included with every request to the service. It must be set to the value of the set-cookie header in the response received to the POST /authenticate/ call."
        } ]
      }
    }
  },
  
  .
  .
  .
  



    "Instance-response" : {
      "properties" : {
        "account" : {
          "type" : "string",
          "description" : "Shows the default account for your identity domain."
        },
        "attributes" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "A dictionary of attributes to be made available to the instance. A value with the key \"userdata\" will be made available in an EC2-compatible manner."
        },
        "availability_domain" : {
          "type" : "string",
          "description" : "The availability domain the instance is in"
        },
        "boot_order" : {
          "items" : {
            "type" : "integer"
          },
          "type" : "array",
          "description" : "Boot order list."
        },
        "desired_state" : {
          "type" : "string",
          "description" : "Desired state for the instance. The value can be <code>shutdown</code> or <code>running</code> to shutdown an instance or to restart a previously shutdown instance respectively."
        },
        "disk_attach" : {
          "type" : "string",
          "description" : "A label assigned by the user to identify disks."
        },
        "domain" : {
          "type" : "string",
          "description" : "The default domain to use for the hostname and for DNS lookups."
        },
        "entry" : {
          "type" : "integer",
          "description" : "Optional imagelistentry number (default will be used if not specified)."
        },
        "error_reason" : {
          "type" : "string",
          "description" : "The reason for the instance going to error state, if available."
        },
        "fingerprint" : {
          "type" : "string",
          "description" : "SSH server fingerprint presented by the instance."
        },
        "hostname" : {
          "type" : "string",
          "description" : "The hostname for this instance."
        },
        "hypervisor" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "A dictionary of hypervisor-specific attributes."
        },
        "image_format" : {
          "type" : "string",
          "description" : "The format of the image."
        },
        "imagelist" : {
          "type" : "string",
          "description" : "Name of imagelist to be launched."
        },
        "ip" : {
          "type" : "string",
          "description" : "IP address of the instance."
        },
        "label" : {
          "type" : "string",
          "description" : "A label assigned by the user, specifically for defining inter-instance relationships."
        },
        "name" : {
          "type" : "string",
          "description" : "Multipart name of the instance."
        },
        "networking" : {
          "additionalProperties" : {
            "type" : "object"
          },
          "type" : "object",
          "description" : "Mapping of <device name> to network specifiers for virtual NICs to be attached to this instance."
        },
        "placement_requirements" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "A list of strings specifying arbitrary tags on nodes to be matched on placement."
        },
        "platform" : {
          "type" : "string",
          "description" : "The OS platform for the instance."
        },
        "priority" : {
          "type" : "string",
          "description" : "The priority at which this instance will be run."
        },
        "quota" : {
          "type" : "string",
          "description" : "Not used"
        },
        "relationships" : {
          "items" : {
            "additionalProperties" : {
              "type" : "object"
            },
            "type" : "object"
          },
          "type" : "array",
          "description" : "A list of relationship specifications to be satisfied on this instance's placement"
        },
        "resolvers" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "Resolvers to use instead of the default resolvers."
        },
        "reverse_dns" : {
          "type" : "boolean",
          "description" : "Add PTR records for the hostname."
        },
        "shape" : {
          "type" : "string",
          "description" : "A shape is a resource profile that specifies the number of CPU threads and the amount of memory (in MB) to be allocated to an instance."
        },
        "sshkeys" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "SSH keys that will be exposed to the instance."
        },
        "start_time" : {
          "type" : "string",
          "description" : "Start time of the instance."
        },
        "state" : {
          "type" : "string",
          "description" : "State of the instance."
        },
        "storage_attachments" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "List of dictionaries containing storage attachment Information."
        },
        "tags" : {
          "items" : {
            "type" : "string"
          },
          "type" : "array",
          "description" : "Comma-separated list of strings used to tag the instance."
        },
        "uri" : {
          "type" : "string",
          "description" : "Uniform Resource Identifier"
        },
        "vcable_id" : {
          "type" : "string",
          "description" : "vCable for this instance."
        },
        "vnc" : {
          "type" : "string",
          "description" : "IP address and port of the VNC console for the instance."
        }
      }
    },

</pre>



In [ ]:
type(operation_details)
print(operation_details.keys())

#### Insert results in to DB

In [ ]:
instances_db.insert(operation_details)

In [ ]:
instances_db.all()[-1]